In [90]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
nltk.download('punkt')
nltk.download('stopwords')

# Define a list of related questions for each query
related_questions = {
    
    'what':[
          "What is a visa ?",
          "Why do I need a visa ?",
          "What types of visas are there ?",
          "What is the purpose of a visa ?"
    ],
    'canada':
    {
      "how": [
          "What documents do I need to apply for a Canadian visa?",
          "What is the processing time for a Canadian visa?",
          "What are the visa fees for Canada?",
          "Can I apply for a Canadian visa online?",
          "What are the requirements for a long-term Canadian visa?"
      ],
      "visa requirements": [ #general
          "What documents do I need to apply for a Canadian visa?",
          "How long does it take to process a Canadian visa?",
          "What is the cost of a Canadian visa?",
          "What are the different types of Canadian visas?"
      ],
      "student": [
          "What are the requirements for a student visa in Canada?",
          "How do I apply for a student visa in Canada?",
          "Can I work in Canada on a student visa?",
          "What is the length of stay for a student visa in Canada?"
      ],
      "tourist": [
        "What are the requirements for a tourist visa in Canada?",
        "How do I apply for a tourist visa in Canada?",
        "What is the length of stay for a tourist visa in Canada?",
        "Can I extend my tourist visa in Canada?"
      ],
      'documents': [
          'What documents are required for a visa application?',
          'How do I submit my visa application?',
          'What is the visa application fee?',
          'What should I do if I am missing a required document for my visa application?',
          'Do I need to get my documents translated for a Canadian visa?',
          'Can I submit copies of my documents for a Canadian visa application?',
          'How long does it take to get the required documents for a Canadian visa?',
          'What are the common mistakes to avoid when preparing my visa application documents?'
      ],
      'fees': [
          'How much does a Canadian visa cost?',
          'What is the visa application fee?',
          'Are there any additional fees for a Canadian visa?',
          'Can I get a refund if my Canadian visa application is rejected?',
          'What are the accepted methods of payment for Canadian visa fees?',
          'How do I pay the Canadian visa application fee from another country?',
          'Are there any discounts or exemptions for Canadian visa fees?',
          'How often do the Canadian visa fees change?'
      ],
      'time': [
          'What is the processing time for a Canadian visa?',
          'How long does it take to get a Canadian visa?',
          'Can I expedite my Canadian visa application?',
          'What is the best time to apply for a Canadian visa?',
          'How long can I stay in Canada with a visa?',
          'Can I enter Canada before my visa starts?',
          'What should I do if my visa expires while I am in Canada?',
          'How far in advance should I apply for a Canadian visa?'
      ]

    },


    'france':
    {
      "how": [
          "What documents do I need to apply for a French visa?",
          "What is the processing time for a French visa?",
          "What are the visa fees for France?",
          "Can I apply for a French visa online?",
          "What are the requirements for a long-term French visa?"
      ],
      "visa requirements": [ #general
          "What documents do I need to apply for a French visa?",
          "How long does it take to process a French visa?",
          "What is the cost of a French visa?",
          "What are the different types of French visas?",
      ],
      "student": [
          "What are the requirements for a student visa in France?",
          "How do I apply for a student visa in France?",
          "Can I work in France on a student visa?",
          "What is the length of stay for a student visa in France?",
      ],
      "tourist": [
          "What are the requirements for a tourist visa in France?",
          "How do I apply for a tourist visa in France?",
          "What is the length of stay for a tourist visa in France?",
          "Can I extend my tourist visa in France?",
      ],
      
      'documents': [
          'What documents are required for a visa application?',
          'How do I submit my visa application?',
          'What is the visa application fee?',
          'What should I do if I am missing a required document for my visa application?',
          'Do I need to get my documents translated for a French visa?',
          'Can I submit copies of my documents for a French visa application?',
          'How long does it take to get the required documents for a French visa?',
          'What are the common mistakes to avoid when preparing my visa application documents?'
      ],
      'fees': [
          'How much does a French visa cost?',
          'What is the visa application fee?',
          'Are there any additional fees for a French visa?',
          'Can I get a refund if my French visa application is rejected?',
          'What are the accepted methods of payment for French visa fees?',
          'How do I pay the French visa application fee from another country?',
          'Are there any discounts or exemptions for French visa fees?',
          'How often do the French visa fees change?'
      ],
      'time': [
          'What is the processing time for a French visa?',
          'How long does it take to get a French visa?',
          'Can I expedite my French visa application?',
          'What is the best time to apply for a French visa?',
          'How long can I stay in France with a visa?',
          'Can I enter France before my visa starts?',
          'What should I do if my visa expires while I am in France?',
          'How far in advance should I apply for a French visa?'
      ]
    }
}
keyword_links = {
    'documents': [
        {'title': 'Canada Visitor Visa Documents', 'url': 'https://www.canada.ca/en/immigration-refugees-citizenship/services/visit-canada/documents-visitor-visa.html'},
        {'title': 'Document Checklist for Canada Tourist Visa', 'url': 'https://www.visa4you.info/canada-tourist-visa-document-checklist/'},
        {'title': 'Canada Visa Document Requirements', 'url': 'https://www.visahq.ca/canada/requirements/'},
        {'title': 'Canada Student Visa Documents', 'url': 'https://www.canada.ca/en/immigration-refugees-citizenship/services/study-canada/study-permit/prepare/get-documents.html'}
    ],
    'fees': [
        {'title': 'Canada Visitor Visa Fees', 'url': 'https://www.canada.ca/en/immigration-refugees-citizenship/services/visit-canada/fees.html'},
        {'title': 'Canada Visa Fees and Costs', 'url': 'https://www.visahq.ca/canada/fees/'},
        {'title': 'Canada Student Visa Fees', 'url': 'https://www.canada.ca/en/immigration-refugees-citizenship/services/study-canada/study-permit/prepare/get-fees.html'}
    ],
    'time': [
        {'title': 'Canada Visitor Visa Processing Time', 'url': 'https://www.canada.ca/en/immigration-refugees-citizenship/services/application/check-processing-times.html'},
        {'title': 'How Long Does It Take to Get a Canadian Visa?', 'url': 'https://www.visaplace.com/blog-immigration-law/canadian-visa-application/how-long-does-it-take-to-get-a-canadian-visa/'},
        {'title': 'Canada Student Visa Processing Time', 'url': 'https://www.canada.ca/en/immigration-refugees-citizenship/services/application/check-processing-times.html'}
    ],
    'insurance': [
        {'title': 'Travel Insurance for Canada Visa', 'url': 'https://www.visitorscoverage.com/canadian-visa-insurance/canadian-visa-insurance/'}      
    ]
}
sponsoredLink = {
    'travel':
    [
      { 'url':'https://www.makemytrip.com/'}
    ],
    'stay':
    [
         {'url':'https://www.trivago.in/'}
    ],
    'insurance':[
        {'url':'https://www.policybazaar.com/travel-insurance/'}
    ]
}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
# Define a function to recommend website links based on a user's query
def recommend_links(query):
    # Define a dictionary of keywords and their corresponding website links
    
    
    # Extract keywords from query
    stop_words = set(stopwords.words('english'))
    tokenized_query = nltk.word_tokenize(query)
    filtered_query = [w.lower() for w in tokenized_query if not w.lower() in stop_words]
    
    # Find website links matching the keywords
    recommended_links = []
    for keyword in keyword_links.keys():
        if keyword in filtered_query:
            recommended_links.extend(keyword_links[keyword])
    
    # Format recommended links as text response
    if recommended_links:
        response = ""
        for link in recommended_links:
            response += f"{link['url']} "
            break
    else:
        return "Sorry, I couldn't find any relevant website links for your query."
    
    return recommended_links[random.randint(0,3)]

def get_related_questions(query):
    words = query.lower().split()
    related = []

    for word in words:
        if word == 'france' or word == 'canada':
            for subcategory in related_questions[word]:
                related += related_questions[word][subcategory]
       
        elif word in related_questions:
            related += related_questions[word]
            
    l1= list(set(related))
    return l1[random.randint(0,3)] #generates one of 4

get_related_questions('France travel visa and documents required')
def get_sponsored_link(query):
    # Define a dictionary of keywords and their corresponding website links
    
    
    # Extract keywords from query
    stop_words = set(stopwords.words('english'))
    tokenized_query = nltk.word_tokenize(query)
    filtered_query = [w.lower() for w in tokenized_query if not w.lower() in stop_words]
    
    # Find website links matching the keywords
    SP_links = []
    for keyword in sponsoredLink.keys():
        if keyword in filtered_query:
            SP_links.extend(sponsoredLink[keyword])
    
    # Format recommended links as text response
    if SP_links:
        response = ""
        for link in SP_links:
            response += f"{link['url']} "
            break
    else:
        response = "Sorry, I couldn't find any relevant website links for your query."
    
    return response


In [95]:
def chatbot(query):
    # Recommend website links based on query
    link_response = recommend_links(query)
    question_response = get_related_questions(query)
    SonsoredLink=get_sponsored_link(query)
    # Combine link and question responses
    response = f"OfficialLink={link_response}&SonsoredLink={SonsoredLink}&Question={question_response}"
    return response

In [97]:
query="what is applicication process for applying visa to canada"
response = chatbot(query)
# print(response)
my_dict = {}

for item in response.split("&"):
    key, value = item.split("=")
    my_dict[key] = value
print(my_dict)

{'OfficialLink': "Sorry, I couldn't find any relevant website links for your query.", 'SonsoredLink': "Sorry, I couldn't find any relevant website links for your query.", 'Question': 'What are the accepted methods of payment for Canadian visa fees?'}
